** What I want to do: ** Ascertain the accuracy and sensitivity of my predictions based on the validation set.

## Admin stuff

In [1]:
from __future__ import division, print_function

import os, json
from glob import glob
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt
from datetime import datetime
import re

In [2]:
%matplotlib inline

In [3]:
data_dir = "data"
train_path = "data/train/"
test_path = "data/test/"
validation_path = "data/valid/"
sample_train_path = "data/sample/train/"
sample_validation_path = "data/sample/valid/"
results_path = "data/results/"

In [4]:
# some useful utilities and the vgg16 model
import utils; reload(utils)
from utils import plots
from utils import save_array, load_array

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
Using Theano backend.


In [5]:
# import the vgg16 model
import vgg16; reload(vgg16)
from vgg16 import Vgg16, image

## Load model weights

Loading model weights for the 1st epoch. This was the model that got the highest validation accuracy of 51%. Remember this is 10 classes. So accuracy explains only so much.

In [6]:
# instantiate the vgg16 model
vgg = Vgg16()

/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/backend/theano_backend.py:1500: UserWarning: DEPRECATION: the 'ds' parameter is not going to exist anymore as it is going to be replaced by the parameter 'ws'.
  mode='max')
/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/backend/theano_backend.py:1500: UserWarning: DEPRECATION: the 'st' parameter is not going to exist anymore as it is going to be replaced by the parameter 'stride'.
  mode='max')
/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/backend/theano_backend.py:1500: UserWarning: DEPRECATION: the 'padding' parameter is not going to exist anymore as it is going to be replaced by the parameter 'pad'.
  mode='max')


In [7]:
vgg.model.load_weights(results_path+'ft_1epoch.h5')

## Run this model against validation set

In [8]:
%time val_batches, val_proba = vgg.test(sample_validation_path, batch_size = 64)

Found 40 images belonging to 10 classes.
CPU times: user 5.93 s, sys: 348 ms, total: 6.28 s
Wall time: 5.78 s


In [30]:
# get true labels
true_labels = val_batches.classes
true_labels.shape

(40,)

In [16]:
# get filenames for each validation set
filenames = val_batches.filenames
filenames[:10]

['c0/img_14395.jpg',
 'c0/img_58938.jpg',
 'c0/img_63398.jpg',
 'c0/img_71758.jpg',
 'c1/img_16267.jpg',
 'c1/img_43562.jpg',
 'c1/img_6407.jpg',
 'c1/img_75390.jpg',
 'c2/img_34137.jpg',
 'c2/img_64545.jpg']

In [25]:
val_proba.shape

(40, 10)

In [29]:
# taking a look at the first 2. We know that their tru labels are c0
val_proba[0:2,:]

array([[  2.2915e-02,   8.2373e-04,   4.3720e-07,   2.7650e-04,   7.6341e-01,   1.3645e-01,
          2.1882e-05,   6.5081e-02,   1.1398e-03,   9.8767e-03],
       [  7.3660e-01,   1.1655e-07,   1.9237e-03,   6.9539e-04,   1.2610e-02,   4.3018e-06,
          6.3679e-05,   1.5895e-02,   2.3193e-01,   2.7473e-04]], dtype=float32)

Remember from pre-processing that the submission requires all 10 probabilities. So this answer is in a pretty good shape. Not much pre-processing (of the predictions) to be done before submission.

In [23]:
!head data/sample_submission.csv

img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
img_1.jpg,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
img_10.jpg,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
img_100.jpg,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
img_1000.jpg,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
img_100000.jpg,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
img_100001.jpg,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
img_100002.jpg,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
img_100003.jpg,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
img_100004.jpg,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1


Given this new information, I now need to change the tru labels so that we can compare apples to apples.

In [32]:
true_labels[:10]

array([0, 0, 0, 0, 1, 1, 1, 1, 2, 2], dtype=int32)

In [36]:
# converting into one-hot
nb_classes=10
true_labels_one_hot = np.eye(nb_classes)[true_labels]
true_labels_one_hot.shape

(40, 10)

## Assess performance of the model

Confusion matrix is not the best indicator for accuracy here. I can manually calculate the multi-class log loss, but what next? Best to simply submit. Assess performance for future models.